# Jakobi 11

In [2]:
from classes import *
import pandas as pd
import numpy as np
import os

pathToFile = os.path.abspath(os.curdir)
df_line_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Line_data")
df_bus_data = pd.read_excel(f"{pathToFile}\\files\\network_configuration.xlsx", sheet_name="Bus_data")
line_data = df_line_data.to_dict(orient='records')
bus_data = df_bus_data.to_dict(orient='records')
line_admittences = []
num_buses = len(bus_data)

for line_dict in line_data:
    line = Line(
        bus_p=line_dict['From line'],
        bus_q=line_dict['To line'],
        impedance=complex(line_dict['R[pu]'], line_dict['X[pu]']),
        half_line_charging_admittance=complex(line_dict['Half Line Charging Admittance'])
    )
    line_admittences.append(line.build_Line_Y_bus())

# Initialize YBUS for a N-Bus System
Ybus_init = Y_Bus(line_adm=line_admittences)
YBus = Ybus_init.build_YBUS(num_buses=num_buses)
display('YBus', YBus)


display('bus_data', df_bus_data)

'YBus'

,1,2,3,4,5
1,6.305-18.750j,-5.000000+15.000000j,-1.250000+ 3.750000j,0.000000+ 0.000000j,0.00+ 0.00j
2,-5.000+15.000j,10.918333-32.500000j,-1.666667+ 5.000000j,-1.666667+ 5.000000j,-2.50+ 7.50j
3,-1.250+ 3.750j,-1.666667+ 5.000000j,12.971667-38.750000j,-10.000000+30.000000j,0.00+ 0.00j
4,0.000+ 0.000j,-1.666667+ 5.000000j,-10.000000+30.000000j,12.971667-38.750000j,-1.25+ 3.75j
5,0.000+ 0.000j,-2.500000+ 7.500000j,0.000000+ 0.000000j,-1.250000+ 3.750000j,3.79-11.25j


'bus_data'

,Bus Code,Assumed bus voltage (pu),Angle,Generation (MW),Generation (MVAr),Load (MW),Load (MVAr)
0,1,1.06,0,-,-,0,0
1,2,1.00,0,40,-,20,10
2,3,1.00,0,0,0,45,15
3,4,1.00,0,0,0,40,5
4,5,1.00,0,0,0,60,10


In [3]:
import math
import numpy as np

##############3 Dette er midlertidig. Får fra Andreas.###################################################
list_dict_power = [{'P1': False, 'P2': True, 'P3': True, 'P4': True, 'P5': True},
                  {'Q1': False, 'Q2': False, 'Q3': True, 'Q4': True, 'Q5': True},
                  {'dirac1': False, 'dirac2': True, 'dirac3': True, 'dirac4': True, 'dirac5': True}]
#########################################################################################################



def J11_calc(bus_data, YBus, list_dict_power): # Mixed form

    row_count = 0
    col_count = 0

    for element in list_dict_power:
        for key, value in element.items():                          # Teller P'er og dirac'er for å vite størrelsen på J11
            if key.startswith('P') and value is True:
                row_count += 1               
            elif key.startswith('d') and value is True:
                col_count += 1           
    df_J11 = pd.DataFrame(0, index=range(1, row_count+1), columns=range(1, col_count+1), dtype=complex) # Lager en tom dataframe med complexe verdier lik 0


# Iterate through each row and column of the J11 array

    for i in range(1, row_count+1):
        for j in range(1, col_count+1):
            if i == j:
                continue
            
            # Tilordner variabler
            vi = abs(bus_data[i]['Assumed bus voltage (pu)'])
            vj = abs(bus_data[j]['Assumed bus voltage (pu)'])
            gij = YBus.at[i, j].real
            sin_ij = math.sin(bus_data[i]['Angle'] - bus_data[j]['Angle'])
            bij = YBus.at[i, j].imag
            cos_ij = math.cos(bus_data[i]['Angle'] - bus_data[j]['Angle'])

            # Beregner elementet i matrisen
            df_J11.at[i, j] = vi * vj *(gij *sin_ij - bij * cos_ij)

    return df_J11
            
test = J11_calc(bus_data=bus_data,
         YBus=YBus,
         list_dict_power=list_dict_power)

print(test)


              1           2             3           4
1   0.00+ 0.00j -15.0+ 0.0j  -3.75+ 0.00j   0.0+ 0.0j
2 -15.00+ 0.00j   0.0+ 0.0j  -5.00+ 0.00j  -5.0+ 0.0j
3  -3.75+ 0.00j  -5.0+ 0.0j   0.00+ 0.00j -30.0+ 0.0j
4   0.00+ 0.00j  -5.0+ 0.0j -30.00+ 0.00j   0.0+ 0.0j


In [6]:
# Calculate mismatches

temp_jacobian = np.random.uniform(-20, 20, size=(7, 7))
temp_jacobian_pd = pd.DataFrame(temp_jacobian)
print(temp_jacobian_pd)                                     # df just for visualization 


# Delta P2 = P2 specified - P2 calculated







           0         1          2          3          4          5          6
0 -19.746307 -2.803398   8.785468   8.209827 -13.082511 -16.309587 -14.342401
1  19.016689 -4.414106 -12.775664  16.901152   8.226690  12.356062 -16.684716
2  -3.972730  5.697673  -5.349554  14.318949  11.371996 -16.190255  -1.984906
3  -9.936174  0.089971   3.222244   4.847187  -8.507862 -16.134588  -9.915863
4 -14.318562  5.947259   2.994160  -3.360276  13.389760  -1.519347 -19.143225
5 -18.527906 -7.914319  -5.492765 -14.961446  15.511807 -11.655478  11.181043
6   0.488569  6.682180  12.173699   9.008885 -15.109407  -0.903275  -4.478536
